In [2]:
import os
import sys
import librosa
import tflearn
import tensorflow as tf
import librosa.display
import IPython.display
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as ms
%matplotlib inline

curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Colocations handled automatically by placer.


In [3]:
os.chdir('C:/Users/karti/Speech/LibriSpeech/dev-clean/')

In [4]:
new_y=[]
f = open("y.txt", "r")
for x in f:
  new_y.append(int(x.strip()))

In [5]:
len(new_y)

2703

In [6]:
new_y = np.array(new_y).reshape(-1,1)

In [7]:
# import pickle
# with open('mfccs.pkl', 'wb') as f:
#     pickle.dump(mfccs, f)

In [8]:
# from sklearn import preprocessing
# le = preprocessing.LabelEncoder()
# le.fit(new_y)
# res=le.transform(new_y)

In [9]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(new_y)
res=enc.transform(new_y).toarray()

In [10]:
res[0]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.])

In [11]:
import pickle
new_mfccs=[]
with open('mfccs.pkl', 'rb') as f:
    new_mfccs = pickle.load(f)

In [12]:
len(new_mfccs)

2703

In [13]:
uni=[]
for i in new_y:
    if(i not in uni):
        uni.append(i)

In [14]:
len(uni)

40

In [15]:
print((new_mfccs[0][0]).mean())

-320.3857


In [16]:
len(new_mfccs)

2703

In [17]:
new_mfccs[200]

array([[-422.61273   , -420.83978   , -421.63156   , ..., -474.38013   ,
        -474.8237    , -455.48975   ],
       [  47.226917  ,   46.655884  ,   49.06836   , ...,   48.494804  ,
          46.773216  ,   48.03399   ],
       [ -28.221333  ,  -29.218151  ,  -22.760973  , ...,    9.897391  ,
           8.224331  ,  -11.62133   ],
       ...,
       [  -6.1200995 ,   -6.310774  ,   -4.5463758 , ...,   -6.9940376 ,
          -5.34544   ,    3.0775084 ],
       [  10.111799  ,   13.743115  ,   21.587517  , ...,    0.93357164,
          -2.2579224 ,   -6.3719716 ],
       [   3.061989  ,    9.477272  ,   14.413259  , ...,   11.6231785 ,
           6.7680616 ,    5.625402  ]], dtype=float32)

In [18]:
padding0_mfccs=[]

In [19]:
len(new_mfccs[50][0])

464

In [20]:
# find max
min_len=len(new_mfccs[0][0])
t=0
for i in new_mfccs:
    t+=1
    if(len(i[0])<min_len):
        min_len=len(i[0])
print(min_len)

63


In [21]:
# find max
max_len=0
t=0
for i in new_mfccs:
    t+=1
    if(len(i[0])>max_len):
        max_len=len(i[0])
print(max_len)

1406


In [22]:
mid=(min_len+max_len)//2
mid

734

In [23]:
for i in range(len(new_mfccs)):
    fill=[]
    for j in range (len(new_mfccs[i])):
        inner_fill=[]
        loop=min(len(new_mfccs[i][j]), mid);
        for k in range(loop):
            inner_fill.append(new_mfccs[i][j][k])
        for k in range(mid-loop):
            inner_fill.append(0)
        fill.append(inner_fill)
#     print(fill)
    padding0_mfccs.append(fill)
#     print(len(padding0_mfccs[0][0]))
#     break

In [24]:
len(padding0_mfccs[100][0])

734

In [25]:
# import pickle
# with open('padding_gauss_mfccs_lstm.pkl', 'wb') as f:
#     pickle.dump(padding_gaussian_mfccs, f)

In [26]:
res

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padding0_mfccs, res, test_size=0.33, random_state=42)

In [28]:
import numpy as np
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers

In [29]:
X_train=np.array(X_train)
X_train.shape

(1811, 13, 734)

In [30]:
X_test=np.transpose(X_test,(0,2,1))

In [31]:
X_test.shape

(892, 734, 13)

In [32]:
X_train=np.transpose(X_train,(0,2,1))

In [33]:
X_train.shape

(1811, 734, 13)

# Model 3

In [34]:
net_lstm_3 = tflearn.input_data([None, 734, 13])
# net = tflearn.embedding(net, input_dim=10000, output_dim=128)
net_lstm_3 = tflearn.lstm(net_lstm_3, 128, dropout=0.8, return_seq= False)
# net_lstm = tflearn.lstm(net_lstm, 128, dropout=0.4)
net_lstm_3 = tflearn.fully_connected(net_lstm_3,40, activation='softmax')
net_lstm_3 = tflearn.regression(net_lstm_3, optimizer='adam', learning_rate=0.001,
                         loss='categorical_crossentropy')

# Training

Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [35]:
model3_lstm = tflearn.DNN(net_lstm_3, tensorboard_verbose=0)

Instructions for updating:
Use tf.cast instead.


In [36]:
model3_lstm.fit(X_train, y_train, n_epoch=1000, show_metric=True, batch_size=128,snapshot_step=100)

Training Step: 14999  | total loss: 0.51518 | time: 33.558s
| Adam | epoch: 1000 | loss: 0.51518 - acc: 0.9388 -- iter: 1792/1811
Training Step: 15000  | total loss: 0.46649 | time: 36.495s
| Adam | epoch: 1000 | loss: 0.46649 - acc: 0.9449 -- iter: 1811/1811
--


In [43]:
model3_lstm.fit(X_train, y_train, n_epoch=100, show_metric=True, batch_size=128,snapshot_step=100)

Training Step: 16499  | total loss: 0.35746 | time: 52.497s
| Adam | epoch: 1100 | loss: 0.35746 - acc: 0.9566 -- iter: 1792/1811
Training Step: 16500  | total loss: 0.32386 | time: 56.690s
| Adam | epoch: 1100 | loss: 0.32386 - acc: 0.9609 -- iter: 1811/1811
--


In [49]:
y_pred = model3_lstm.predict(X_train)

In [50]:
np.sum(np.argmax(y_train, axis=1)==np.argmax(y_pred, axis=1))/len(y_train)

0.9591385974599669

In [51]:
y_pred = model3_lstm.predict(X_test)

In [52]:
np.sum(np.argmax(y_test, axis=1)==np.argmax(y_pred, axis=1))/len(y_test)

0.9091928251121076

In [42]:
model3_lstm.save('model3_0_LSTM_128.tflearn')

INFO:tensorflow:C:\Users\karti\Speech\LibriSpeech\dev-clean\model3_0_LSTM_128.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [48]:
model3_lstm.load('model3_0_LSTM_128.tflearn')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\karti\Speech\LibriSpeech\dev-clean\model3_0_LSTM_128.tflearn
